In [2]:
import torch
import torch.nn.functional as f
import torch.nn as nn
import cv2
import os
import numpy as np
from tqdm import tqdm
import torch.optim as optim

In [3]:
create_data=False
convert_data=False
model_exists=False

In [4]:
if create_data:
    cap=cv2.VideoCapture(0)
    file='data\\left'
    i=0
    while i<500:
        name='left'
        name=name+str(i)+'.jpg'
        i+=1
        ret,frame=cap.read()
        try:
            cv2.imshow('frame',frame)
        except:
            pass

        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
        p=os.path.join(file,name)
        cv2.imwrite(p,frame)

    cap.release()
    cv2.destroyAllWindows()

In [5]:
if convert_data:
    data_file='data'
    labels=['right','left','idle']
    data=[]
    for label in labels:
        path=os.path.join(data_file,label)
        for file in tqdm(os.listdir(path)):
            try:
                image=cv2.imread(os.path.join(path,file))
                image=cv2.resize(image,(50,50))
                image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                d=np.array([image,label])
                data.append(d)
            except:
                print('h')
    data=np.array(data)
    np.random.shuffle(data)
    np.save('data.npy',data)

In [6]:
data=np.load('data.npy',allow_pickle=True)
cv2.imshow('pic',data[2][0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
test_size=int(data.shape[0]*0.1)
training_data=data[:-test_size]
test_data=data[-test_size:]
actions={'left':0,'right':1,'idle':2}
dc={'dog':0,'cat':1}

In [8]:
training_x=[]
training_y=[]
test_set=[]
for x in training_data:
    training_x.append(x[0]/255)
#     print(x[1])
#     training_y.append(x[1])
    training_y.append(np.eye(3)[actions[x[1]]])
training_x=torch.Tensor(training_x)
# print(training_y)
training_y=torch.Tensor(training_y)

for x in test_data:
    k=np.array([x[0],np.eye(3)[actions[x[1]]]])
    test_set.append(k)
# test_set

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,50,5)
        self.conv2=nn.Conv2d(50,100,5)
        self.conv3=nn.Conv2d(100,200,5)
        self.flat=-1
        
        self.flatten()
        
        self.fc1=nn.Linear(self.flat,self.flat)
        self.fc2=nn.Linear(self.flat,100)
        self.fc3=nn.Linear(100,3)
    
    def flatten(self):
        x=np.random.rand(50,50)
#         print(x)
        x=torch.Tensor(x).view(-1,1,50,50)
        x=(f.max_pool2d(self.conv1(x),(2,2)))
        x=(f.max_pool2d(self.conv2(x),(2,2)))
        x=(f.max_pool2d(self.conv3(x),(2,2)))
#         print(x.shape)
        self.flat=x.shape[1]*x.shape[2]*x.shape[3]
    
    def forward(self,x):
        x=f.relu(f.max_pool2d(self.conv1(x),(2,2)))
        x=f.relu(f.max_pool2d(self.conv2(x),(2,2)))
        x=f.relu(f.max_pool2d(self.conv3(x),(2,2)))
        x=f.relu(self.fc1(x.view(-1,self.flat)))
        x=f.relu(self.fc2(x))
        x=f.relu(self.fc3(x))
#         print(x)
        return x

In [10]:
torch.cuda.is_available()

True

In [11]:
device=torch.device('cuda:0')
device

device(type='cuda', index=0)

In [12]:
net=Net().to(device)

In [13]:
x=np.random.rand(50,50)
print(x)
x=torch.Tensor(x).view(-1,1,50,50).to(device)
y=net.forward(x)

[[0.08604913 0.97304405 0.2938683  ... 0.1098536  0.71497969 0.3464271 ]
 [0.49135731 0.82080028 0.74050852 ... 0.68763181 0.65733556 0.62742797]
 [0.94349731 0.86496495 0.9150428  ... 0.93833072 0.88621613 0.73920402]
 ...
 [0.07866425 0.22788643 0.82138235 ... 0.65496799 0.81955757 0.84022921]
 [0.3586896  0.52371193 0.74841534 ... 0.47844935 0.22991434 0.54340015]
 [0.18809612 0.4158535  0.76346885 ... 0.56049785 0.78805604 0.59185474]]


In [18]:
optimizer=optim.Adam(net.parameters(),lr=0.001)
loss_func=nn.MSELoss()
epochs=3
batch_size=100
for epoch in range(epochs):
    for i in tqdm(range(0,training_data.shape[0],batch_size)):
        net.zero_grad()
        x=training_x[i:i+batch_size].to(device)
        y=training_y[i:i+batch_size].to(device)
        output=net(x.view(-1,1,50,50))
        loss=loss_func(output,y)
        loss.backward()
        optimizer.step()
    print(loss)


100%|██████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 24.96it/s]


tensor(0.1102, device='cuda:0', grad_fn=<MseLossBackward>)


100%|██████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 20.26it/s]


tensor(0.1079, device='cuda:0', grad_fn=<MseLossBackward>)


100%|██████████████████████████████████████████████████████████████████| 14/14 [00:01<00:00, 11.55it/s]


tensor(0.0347, device='cuda:0', grad_fn=<MseLossBackward>)


In [43]:
correct=0
total=0
for i in test_set:
    x=torch.Tensor(i[0]/255).view(-1,50,50).to(device)
    y=torch.argmax(torch.Tensor(i[1])).to(device)
    output=net(x.view(-1,1,50,50))
    if y==torch.argmax(output):
        correct+=1
    total+=1
print(correct/total)

1.0


In [47]:
cap=cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    ret,frame=cap.read()
    x=cv2.resize(frame,(50,50))
    x=cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    x=x/255
    x=torch.Tensor(x).view(-1,1,50,50).to(device)
    output=torch.argmax(net(x))
    cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
    if output==1:
        cv2.putText(frame,'right',(10,200), font, 1, (200,255,123), 3, cv2.LINE_AA)
#         print('right')
    elif output==2:
        pass
    else:
#         print('left')
        cv2.putText(frame,'left',(550,200), font, 1, (200,32,155), 3, cv2.LINE_AA)
    if cv2.waitKey(300) & 0xFF == ord('q'):
        break
    cv2.imshow('frame',frame)
cap.release()
cv2.destroyAllWindows()

In [6]:
cap=cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    ret,frame=cap.read()
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
    cv2.imshow('frame',frame)
cap.release()
cv2.destroyAllWindows()